In [1]:
import os
import json
import re

In [3]:
def merge_details(details):
    merged_list = []
    current_item = ""

    for i, item in enumerate(details):
        if item.strip() and item.strip()[0].isdigit():
            j = 1
            while j < len(item.strip()) and item.strip()[j].isdigit():
                j += 1
            if j < len(item.strip()) and item.strip()[j] == "：":
                merged_list.append(current_item)
                current_item = item.strip()
            else:
                current_item += item.strip()
        else:
            current_item += item.strip()

    merged_list.append(current_item)
    merged_list.pop(0)
    return merged_list


def extract_tuples(string):
    tuples = []
    depth = 0
    start = None
    for i, char in enumerate(string):
        if char == "(" or char == "（":
            if depth == 0:
                start = i
            depth += 1
        elif char == ")" or char == "）":
            depth -= 1
            if depth == 0 and start is not None:
                tuples.append(string[start+1:i])
                start = None
    return tuples


def get_results(merged_list, input_list, rel_extend_map, mode):
    rel_extend_index = 0
    cur_rel_extend_list = []
    A = []

    for idx, item in enumerate(merged_list):
        B = []
        cur_output_3 = item.split('Output:')
        cur_idx = cur_output_3[0].split('：')[0]
        cur_true_rel = input_list[int(cur_idx)-1]['rel_type']
            
        if rel_extend_index == len(cur_rel_extend_list):
            rel_extend_index = 0

        cur_rel_extend_list = rel_extend_map[cur_true_rel]
        if not cur_rel_extend_list:
            continue
        try:
            cur_extend_rel = cur_rel_extend_list[rel_extend_index]
        except:
            print(idx, cur_true_rel)

        for output in cur_output_3[1:]:
            sep = ""
            if 'zh' in mode:
                if "生成关系列表：" in output:
                    sep = "生成关系列表："
                elif "生成的关系列表：" in output:
                    sep = "生成的关系列表："
                elif "生成关系列表为：" in output:
                    sep = "生成关系列表为："
                elif "生成的关系列表为：" in output:
                    sep = "生成的关系列表为："
                elif "生成以下关系列表：" in output:
                    sep = "生成以下关系列表："
            elif 'en' in mode:
                if "Generated list of relations:" in output:
                    sep = "Generated list of relations:"
                elif "Generate a list of relations:" in output:
                    sep = "Generate a list of relations:"
                elif "Generate a list of relations based on the judgment sentence:" in output:
                    sep = "Generate a list of relations based on the judgment sentence:"
                elif "List of relations:" in output:
                    sep = "List of relations:"
                elif "Generate a list of relationships:" in output:
                    sep = "Generate a list of relationships:"
                # elif "subject-object pair:" in output:
                #     sep = "subject-object pair:"
                # elif "Subject-object pair:" in output:
                #     sep = "Subject-object pair:"

            if sep:
                tmp = output.split(sep)[1]
                matches = extract_tuples(tmp)
                if matches:
                    for match in matches:
                        if 'zh' in mode:
                            triple = tuple(match.replace('，',',').strip().strip('（）').split(','))
                        elif 'en' in mode:
                            triple = tuple(match.strip().strip('()').split(','))
                        if len(triple) == 3: 
                            if 'zh' in mode:
                                if cur_extend_rel == triple[0].strip().strip('()').strip('""'):
                                    B.append((int(cur_idx)-1, cur_extend_rel, triple[1].strip().strip('()'), triple[2].strip().strip('()')))
                                elif cur_extend_rel == triple[1].strip().strip('()').strip('""'):
                                    B.append((int(cur_idx)-1, cur_extend_rel, triple[0].strip().strip('()'), triple[2].strip().strip('()')))
                                else:
                                    B.append((int(cur_idx)-1, cur_extend_rel, '', ''))
                            elif 'en' in mode:
                                if sep != "subject-object pair:" and sep!="Subject-object pair:":
                                    B.append((int(cur_idx)-1, cur_extend_rel, triple[0].strip().strip('()'), triple[2].strip().strip('()')))
                                else:
                                    B.append((int(cur_idx)-1, cur_extend_rel, '', ''))
                        elif len(triple) == 2:
                            if sep == "subject-object pair:" or sep == "Subject-object pair:":
                                B.append((int(cur_idx)-1, cur_extend_rel, triple[0].strip().strip('()'), triple[1].strip().strip('()')))
                            else:
                                B.append((int(cur_idx)-1, cur_extend_rel, '', ''))
                        else:
                            B.append((int(cur_idx)-1, cur_extend_rel, '', ''))

                else:
                    B.append((int(cur_idx)-1, cur_extend_rel, '', ''))
                
            else:
                B.append((int(cur_idx)-1, cur_extend_rel, '', ''))
                continue
        if B:
            A.append(B)    

        rel_extend_index += 1

    A = [list(set(item)) for item in A]
    
    with open(f'./results/test_{mode}.txt','w',encoding='utf-8') as f:
        for item in A:
            f.write(str(item)+'\n')

CMeIE

In [4]:
with open('./results/details_zh.txt','r',encoding='utf-8') as f:
    test_details = f.readlines()

with open('./data/cmeie/input_list.json','r',encoding='utf-8') as f:
    input_list = json.load(f)

with open('./data/cmeie/rel_extend_map.json','r',encoding='utf-8') as f:
    rel_extend_map = json.load(f)

merged_list = merge_details(test_details)
get_results(merged_list, input_list, rel_extend_map, 'zh')

SCIERC

In [6]:
with open('./results/details_en.txt','r',encoding='utf-8') as f:
    test_details = f.readlines()

with open('./data/scierc/input_list.json','r',encoding='utf-8') as f:
    input_list = json.load(f)

with open('./data/scierc/rel_extend_map.json','r',encoding='utf-8') as f:
    rel_extend_map = json.load(f)

merged_list = merge_details(test_details)
get_results(merged_list, input_list, rel_extend_map, 'en')

Baichuan2-CMeIE

In [47]:
with open('./results/details_baichuan2_13B_zh.txt','r',encoding='utf-8') as f:
    test_details = f.readlines()

with open('./data/cmeie/input_list.json','r',encoding='utf-8') as f:
    input_list = json.load(f)

with open('./data/cmeie/final_rel_extend_map_baichuan2_13B_zh.json','r',encoding='utf-8') as f:
    rel_extend_map = json.load(f)

merged_list = merge_details(test_details)
get_results(merged_list, input_list, rel_extend_map, 'baichuan2_13B_zh')

Baichuan2-SCIERC

In [46]:
with open('./results/details_baichuan2_13B_en.txt','r',encoding='utf-8') as f:
    test_details = f.readlines()

with open('./data/scierc/input_list.json','r',encoding='utf-8') as f:
    input_list = json.load(f)

with open('./data/scierc/final_rel_extend_map_baichuan2_13B_en.json','r',encoding='utf-8') as f:
    rel_extend_map = json.load(f)

merged_list = merge_details(test_details)
get_results(merged_list, input_list, rel_extend_map, 'baichuan2_13B_en')

Alpaca

In [ ]:
with open('./results/details_alpaca_33B_zh.txt','r',encoding='utf-8') as f:
    test_details = f.readlines()

with open('./data/cmeie/input_list.json','r',encoding='utf-8') as f:
    input_list = json.load(f)

with open('./data/cmeie/final_rel_extend_map_alpaca_33B_zh.json','r',encoding='utf-8') as f:
    rel_extend_map = json.load(f)

merged_list = merge_details(test_details)
get_results(merged_list, input_list, rel_extend_map, 'alpaca_33B_zh')

In [4]:
with open('./results/details_alpaca_33B_en.txt','r',encoding='utf-8') as f:
    test_details = f.readlines()

with open('./data/scierc/input_list.json','r',encoding='utf-8') as f:
    input_list = json.load(f)

with open('./data/scierc/final_rel_extend_map_alpaca_33B_en.json','r',encoding='utf-8') as f:
    rel_extend_map = json.load(f)

merged_list = merge_details(test_details)
get_results(merged_list, input_list, rel_extend_map, 'alpaca_33B_en')